<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/light_20240827.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:

# !pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 10.4 MB/s eta 0:00:00


In [3]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [6]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
#from catboost import CatBoostClassifier, Pool

In [30]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train_allnum.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test_allnum.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [31]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,1.145527e+00,1,0.077180,0,-1.908669,0.361786,1.038068,1,-1.352770,1.0,0,-1.307889,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1.766003e+00,0,-0.067460,0,-1.908669,0.361786,-0.699052,1,0.639729,0.0,1,-1.307889,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,7.347917e-16,1,-0.646021,1,-1.908669,-0.746633,0.459028,0,-1.032001,1.0,0,-1.307889,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,-1.988390e-01,1,0.511101,1,-1.908669,-0.746633,-1.278092,0,-0.388082,0.0,1,-1.307889,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,9.387011e-01,0,0.366461,1,-1.908669,-0.746633,0.459028,0,-1.292825,1.0,1,-1.307889,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
# Config cell
target = "ProdTaken"

In [59]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge',
'IncomeStarInteraction','FollowupEfficiency','IncomePerPerson','StarPerCityTier','IncomePerStar','LuxuryAffinity','FollowupPerTrip']
#numerical_features = default_numerical_features + added_numerical_features
numerical_features = default_numerical_features


#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=[target]
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar']
categorical_features ['TypeofContact', 'Gender', 'Passport', 'ProdTaken', 'HasChildren', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'PreferredPropertyStar_3.0', 'PreferredPropertyStar_4.0', 'PreferredPropertyStar_5.0', 'PitchSatisfactionScore_1', 'PitchSatisfactionScore_2', 'PitchSatisfactionScore_3', 'PitchSatisfactionScore_4', 'PitchSatisfactionScore_5', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatus_未婚', 'MaritalStatus_独身', 'MaritalStatus_結婚', 'MaritalStatus_離婚']
features for training:['id', 'Age', 'TypeofContact', 'DurationOfPitch', 'Gender', 'NumberOfPersonVisiting',

In [56]:
cols_category=categorical_features

In [57]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [39]:


params_lgb = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 20,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': -1,
    'lambda_l1': 0.80,
    'lambda_l2': 90,
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [40]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [58]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
list_feature_importances = []  # 特徴量重要度を格納するリスト

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=520)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]

    # LightGBMデータセットの作成
    train_data = lgb.Dataset(data=trn_x, label=trn_y, categorical_feature=cols_category)
    val_data = lgb.Dataset(data=val_x, label=val_y, categorical_feature=cols_category, reference=train_data)

    model_lgb = lgb.LGBMClassifier(**params_lgb)
    model_lgb.fit(
        trn_x, trn_y,
        eval_set=(val_x, val_y),
        callbacks=[lgb.early_stopping(100, verbose=True)],
        categorical_feature=cols_category,
    )

    list_models.append(model_lgb)
    preds_y_proba = model_lgb.predict_proba(val_x)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, F1 score: {f1}, Threshold: {threshold}")

    # 特徴量の重要度を追加
    feature_importances = model_lgb.feature_importances_
    list_feature_importances.append(feature_importances)

# AUCの出力
print("Mean AUC:", np.mean(list_metrics_auc))
print("Mean F1 Score:", np.mean(list_metrics_f1))
print("Median Cutoff:", np.median(list_cutoff))

# 特徴量重要度の集計
mean_feature_importances = np.mean(list_feature_importances, axis=0)

# 結果の表示
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Mean Importance': mean_feature_importances
}).sort_values(by='Mean Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[674]	valid_0's auc: 0.837709
Fold: 1, AUC: 0.8377090301003345, F1 score: 0.7441214977742865, Threshold: 0.30675062805113396
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[7]	valid_0's auc: 0.844197
Fold: 2, AUC: 0.8441973244147157, F1 score: 0.7180474733537304, Threshold: 0.1452755210151151
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1771]	valid_0's auc: 0.825028
Fold: 3, AUC: 0.8250282457294145, F1 score: 0.7181206936031141, Threshold: 0.28412175032417586
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1214]	valid_0's auc: 0.833881
Fold: 4, AUC: 0.8338813847995818, F1 score: 0.6973246347013052, Threshold: 0.36044536787337617
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2063]	valid_0's

In [ ]:
0.8421905678298216
0.8386405457470276

SyntaxError: invalid syntax (<ipython-input-81-76ea1f53a6c0>, line 1)

In [ ]:
0.8437365146029495

In [ ]:
import pickle

In [ ]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v8{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [ ]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x7c8fdc397d30>, Proba: [0.02020181 0.05508381 0.04953488 ... 0.13518113 0.05969247 0.00122077]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc41959c0>, Proba: [0.0362621  0.09294604 0.10518342 ... 0.25370721 0.11980879 0.00464472]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc4195e40>, Proba: [0.05271098 0.14127187 0.15547861 ... 0.37594977 0.17903203 0.00725419]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fdc6bbaf0>, Proba: [0.06922614 0.18744593 0.20132556 ... 0.50477625 0.23216644 0.00950096]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc4194f10>, Proba: [0.08480226 0.22268625 0.25700218 ... 0.61543423 0.29101282 0.01373879]


In [ ]:
count = 0

In [ ]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)